In [1]:
import torch_geometric
from torch_geometric.datasets import Planetoid

In [2]:
# Load the dataset
dataset = Planetoid(root='E:\Github\Python-self-Learning\PytorchGeometric\lecture_1\cora', name='Cora')

Processing...
Done!


In [3]:
# Print some information about the dataset
print(f'Dataset: {dataset}:')
print('Number of graphs: %d' % len(dataset))
print('Number of clases: %d' % dataset.num_classes)
print('Number of features: %d' % dataset.num_node_features)
print('Number of edge features: %d' % dataset.num_edge_features)

Dataset: Cora():
Number of graphs: 1
Number of clases: 7
Number of features: 1433
Number of edge features: 0


In [5]:
# shapes of the first graph in the dataset
print(dataset[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [6]:
print('edge_index shape: ', dataset[0].edge_index.shape)
print(dataset[0].edge_index[:, :10])
print('\n')
print('train_mask shape: ', dataset[0].train_mask.shape)
print(dataset[0].train_mask[:10])
print('\n')
print('x shape: ', dataset[0].x.shape)
print(dataset[0].x[:10])
print('\n')
print('y shape: ', dataset[0].y.shape)
print(dataset[0].y[:10])

edge_index shape:  torch.Size([2, 10556])
tensor([[   0,    0,    0,    1,    1,    1,    2,    2,    2,    2],
        [ 633, 1862, 2582,    2,  652,  654,    1,  332, 1454, 1666]])


train_mask shape:  torch.Size([2708])
tensor([True, True, True, True, True, True, True, True, True, True])


x shape:  torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y shape:  torch.Size([2708])
tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2])


In [9]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [8]:
data = dataset[0]

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = SAGEConv(dataset.num_node_features, dataset.num_classes, aggr='max')
        
    def forward(self, x, edge_index):
        x = self.conv1(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [11]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [14]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model(data.x, data.edge_index)[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
    
def test():
    model.eval()
    logits, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
        
    return accs

In [20]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 1.0000, Test: 0.7240
Epoch: 020, Val: 1.0000, Test: 0.7240
Epoch: 030, Val: 1.0000, Test: 0.7240
Epoch: 040, Val: 1.0000, Test: 0.7240
Epoch: 050, Val: 1.0000, Test: 0.7240
Epoch: 060, Val: 1.0000, Test: 0.7240
Epoch: 070, Val: 1.0000, Test: 0.7240
Epoch: 080, Val: 1.0000, Test: 0.7240
Epoch: 090, Val: 1.0000, Test: 0.7240
